# 📘 Books Database Project
This project builds a structured books database using a CSV file containing book records with categories, prices, availability, ratings, and more. The database is created using Python, pandas, and MySQBenchwork.

## Imports

In [1]:
import mysql.connector
import pandas as pd
import plotly.express as px

## 1. Load the CSV File

In [7]:
df = pd.read_csv("books.csv")
df

,Category,Title,URL,Price,Availability,Rating
0,Travel,It's Only the Himalayas,http://books.toscrape.com/catalogue/its-only-t...,£45.17,In stock,Two
1,Travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,http://books.toscrape.com/catalogue/full-moon-...,£49.43,In stock,Four
2,Travel,See America: A Celebration of Our National Par...,http://books.toscrape.com/catalogue/see-americ...,£48.87,In stock,Three
3,Travel,Vagabonding: An Uncommon Guide to the Art of L...,http://books.toscrape.com/catalogue/vagabondin...,£36.94,In stock,Two
4,Travel,Under the Tuscan Sun,http://books.toscrape.com/catalogue/under-the-...,£37.33,In stock,Three
...,...,...,...,...,...,...
995,Politics,Why the Right Went Wrong: Conservatism--From G...,http://books.toscrape.com/catalogue/why-the-ri...,£52.65,In stock,Four
996,Politics,Equal Is Unfair: America's Misguided Fight Aga...,http://books.toscrape.com/catalogue/equal-is-u...,£56.86,In stock,One
997,Cultural,Amid the Chaos,http://books.toscrape.com/catalogue/amid-the-c...,£36.58,In stock,One
998,Erotica,Dark Notes,http://books.toscrape.com/catalogue/dark-notes...,£19.19,In stock,Five


In [8]:
# Clean data
rating_map = {"One" : 1 , "Two" : 2 , "Three" : 3 , "Four" : 4 , "Five" : 5}
df['rating'] = df['Rating'].map(rating_map)
df['price'] = df['Price'].str.replace("£", "").astype(float)
df

,Category,Title,URL,Price,Availability,Rating,rating,price
0,Travel,It's Only the Himalayas,http://books.toscrape.com/catalogue/its-only-t...,£45.17,In stock,Two,2,45.17
1,Travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,http://books.toscrape.com/catalogue/full-moon-...,£49.43,In stock,Four,4,49.43
2,Travel,See America: A Celebration of Our National Par...,http://books.toscrape.com/catalogue/see-americ...,£48.87,In stock,Three,3,48.87
3,Travel,Vagabonding: An Uncommon Guide to the Art of L...,http://books.toscrape.com/catalogue/vagabondin...,£36.94,In stock,Two,2,36.94
4,Travel,Under the Tuscan Sun,http://books.toscrape.com/catalogue/under-the-...,£37.33,In stock,Three,3,37.33
...,...,...,...,...,...,...,...,...
995,Politics,Why the Right Went Wrong: Conservatism--From G...,http://books.toscrape.com/catalogue/why-the-ri...,£52.65,In stock,Four,4,52.65
996,Politics,Equal Is Unfair: America's Misguided Fight Aga...,http://books.toscrape.com/catalogue/equal-is-u...,£56.86,In stock,One,1,56.86
997,Cultural,Amid the Chaos,http://books.toscrape.com/catalogue/amid-the-c...,£36.58,In stock,One,1,36.58
998,Erotica,Dark Notes,http://books.toscrape.com/catalogue/dark-notes...,£19.19,In stock,Five,5,19.19


## 2. Connect to MySQL 

In [9]:
con = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'admin',
)

cur = con.cursor()

## 3. Create the books Database

In [10]:
qry = '''
CREATE DATABASE IF NOT EXISTS books
'''

cur.execute(qry)

In [11]:
qry = '''
USE books
'''
cur.execute(qry)

## 4. Create Tables in books Database

In [12]:
# Create categories table
qry = '''
CREATE TABLE IF NOT EXISTS categories (
    category_id INT AUTO_INCREMENT PRIMARY KEY,
    category_name VARCHAR(100) UNIQUE
)
'''
cur.execute(qry)

In [13]:
# Create books_catalog table
qry = '''
CREATE TABLE IF NOT EXISTS books_catalog (
    book_id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    url TEXT,
    price DECIMAL(6,2),
    availability VARCHAR(50),
    rating INT,
    category_id INT,
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
)
'''
cur.execute(qry)
con.commit()

##  5. Insert Categories into categories Table

In [14]:
for cat in df["Category"].unique():
    cur.execute("INSERT IGNORE INTO categories (category_name) VALUES (%s)", (cat,))
con.commit()

In [15]:
#  Create mapping: category_name → category_id
cur.execute("SELECT category_id, category_name FROM categories")
category_map = {name: cid for cid, name in cur.fetchall()}
category_map

{'Academic': 39,
 'Add a comment': 17,
 'Adult Fiction': 28,
 'Art': 24,
 'Autobiography': 26,
 'Biography': 35,
 'Business': 34,
 'Childrens': 10,
 'Christian': 42,
 'Christian Fiction': 33,
 'Classics': 5,
 'Contemporary': 37,
 'Crime': 50,
 'Cultural': 48,
 'Default': 14,
 'Erotica': 49,
 'Fantasy': 18,
 'Fiction': 9,
 'Food and Drink': 32,
 'Health': 46,
 'Historical': 41,
 'Historical Fiction': 3,
 'History': 31,
 'Horror': 30,
 'Humor': 29,
 'Music': 13,
 'Mystery': 2,
 'New Adult': 19,
 'Nonfiction': 12,
 'Novels': 45,
 'Paranormal': 23,
 'Parenting': 27,
 'Philosophy': 6,
 'Poetry': 22,
 'Politics': 47,
 'Psychology': 25,
 'Religion': 11,
 'Romance': 7,
 'Science': 21,
 'Science Fiction': 15,
 'Self Help': 40,
 'Sequential Art': 4,
 'Short Stories': 44,
 'Spirituality': 38,
 'Sports and Games': 16,
 'Suspense': 43,
 'Thriller': 36,
 'Travel': 1,
 'Womens Fiction': 8,
 'Young Adult': 20}

## 6. Insert Books into books_catalog Table

In [2]:
con = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'admin',
    database = 'books'
)

cur = con.cursor()

In [17]:
qry = '''
INSERT INTO books_catalog (title, url, price, availability, rating, category_id)
        VALUES (%s, %s, %s, %s, %s, %s)
'''
for _,row in df.iterrows():
    cur.execute(qry ,
        (row["Title"],
        row["URL"],
        row["price"],
        row["Availability"],
        row["rating"],
        category_map[row["Category"]]))

con.commit()

### Done!Test a Query

In [25]:
qry = '''
SELECT title, price FROM books_catalog;
'''
cur.execute(qry)

for title, price in cur.fetchall():
    print(f"{title} - £{price}\n")

It's Only the Himalayas - £45.17

Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond - £49.43

See America: A Celebration of Our National Parks & Treasured Sites - £48.87

Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel - £36.94

Under the Tuscan Sun - £37.33

A Summer In Europe - £44.34

The Great Railway Bazaar - £30.54

A Year in Provence (Provence #1) - £56.88

The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2) - £23.21

Neither Here nor There: Travels in Europe - £38.95

1,000 Places to See Before You Die - £26.08

Sharp Objects - £47.82

In a Dark, Dark Wood - £19.63

The Past Never Ends - £56.50

A Murder in Time - £16.64

The Murder of Roger Ackroyd (Hercule Poirot #4) - £44.10

The Last Mile (Amos Decker #2) - £54.21

That Darkness (Gardiner and Renner #1) - £13.92

Tastes Like Fear (DI Marnie Rome #3) - £10.69

A Time of Torment (Charlie Parker #14) - £48.35

A Study in Scarlet (Sherlock Holmes #

## You Have Some questions To Answer!

#### 01) Write a sql query to get books that has rate >=3 and has 'Mr' in its name.

In [44]:
qry = '''
SELECT * FROM books_catalog 
WHERE rating >= 3 AND title LIKE '%Mr%';
'''
cur.execute(qry)
cur.fetchall()

[(48,
  'Mrs. Houdini',
  'http://books.toscrape.com/catalogue/mrs-houdini_821/index.html',
  Decimal('30.25'),
  'In stock',
  5,
  3),
 (243,
  'My Mrs. Brown',
  'http://books.toscrape.com/catalogue/my-mrs-brown_719/index.html',
  Decimal('24.48'),
  'In stock',
  3,
  9),
 (248,
  'Dear Mr. Knightley',
  'http://books.toscrape.com/catalogue/dear-mr-knightley_684/index.html',
  Decimal('11.21'),
  'In stock',
  3,
  9),
 (952,
  'Louisa: The Extraordinary Life of Mrs. Adams',
  'http://books.toscrape.com/catalogue/louisa-the-extraordinary-life-of-mrs-adams_818/index.html',
  Decimal('16.85'),
  'In stock',
  3,
  35)]

In [45]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,48,Mrs. Houdini,http://books.toscrape.com/catalogue/mrs-houdin...,30.25,In stock,5,3
1,243,My Mrs. Brown,http://books.toscrape.com/catalogue/my-mrs-bro...,24.48,In stock,3,9
2,248,Dear Mr. Knightley,http://books.toscrape.com/catalogue/dear-mr-kn...,11.21,In stock,3,9
3,952,Louisa: The Extraordinary Life of Mrs. Adams,http://books.toscrape.com/catalogue/louisa-the...,16.85,In stock,3,35


#### 02) Write a sql query to get books that has rate >=3 or price > 20.

In [42]:
qry = '''
SELECT * FROM books_catalog 
WHERE rating >= 3 OR price > 20;
'''
cur.execute(qry)
cur.fetchall()

[(1,
  "It's Only the Himalayas",
  'http://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
  Decimal('45.17'),
  'In stock',
  2,
  1),
 (2,
  'Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond',
  'http://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html',
  Decimal('49.43'),
  'In stock',
  4,
  1),
 (3,
  'See America: A Celebration of Our National Parks & Treasured Sites',
  'http://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html',
  Decimal('48.87'),
  'In stock',
  3,
  1),
 (4,
  'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  'http://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html',
  Decimal('36.94'),
  'In stock',
  2,
  1),
 (5,
  'Under the Tuscan Sun',
  'http://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html',
  Decim

In [43]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,1,It's Only the Himalayas,http://books.toscrape.com/catalogue/its-only-t...,45.17,In stock,2,1
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,http://books.toscrape.com/catalogue/full-moon-...,49.43,In stock,4,1
2,3,See America: A Celebration of Our National Par...,http://books.toscrape.com/catalogue/see-americ...,48.87,In stock,3,1
3,4,Vagabonding: An Uncommon Guide to the Art of L...,http://books.toscrape.com/catalogue/vagabondin...,36.94,In stock,2,1
4,5,Under the Tuscan Sun,http://books.toscrape.com/catalogue/under-the-...,37.33,In stock,3,1
...,...,...,...,...,...,...,...
926,1001,Sapiens: A Brief History of Humankind,#,49.99,In stock,5,54
927,1002,"Guns, Germs, and Steel",#,39.50,In stock,4,54
928,1003,The Silk Roads,#,35.00,In stock,4,54
929,1004,The Wright Brothers,#,28.90,In stock,3,54


#### 03) Write a sql query to get books that has rate not 3.

In [40]:
qry = '''
SELECT * FROM books_catalog 
WHERE rating <> 3;
'''
cur.execute(qry)
cur.fetchall()

[(1,
  "It's Only the Himalayas",
  'http://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
  Decimal('45.17'),
  'In stock',
  2,
  1),
 (2,
  'Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond',
  'http://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html',
  Decimal('49.43'),
  'In stock',
  4,
  1),
 (4,
  'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  'http://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html',
  Decimal('36.94'),
  'In stock',
  2,
  1),
 (6,
  'A Summer In Europe',
  'http://books.toscrape.com/catalogue/a-summer-in-europe_458/index.html',
  Decimal('44.34'),
  'In stock',
  2,
  1),
 (7,
  'The Great Railway Bazaar',
  'http://books.toscrape.com/catalogue/the-great-railway-bazaar_446/index.html',
  Decimal('30.54'),
  'In stock',
  1,
  1),
 (8,
  'A Year in Provence (Provence #1)',
  'h

In [41]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,1,It's Only the Himalayas,http://books.toscrape.com/catalogue/its-only-t...,45.17,In stock,2,1
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,http://books.toscrape.com/catalogue/full-moon-...,49.43,In stock,4,1
2,4,Vagabonding: An Uncommon Guide to the Art of L...,http://books.toscrape.com/catalogue/vagabondin...,36.94,In stock,2,1
3,6,A Summer In Europe,http://books.toscrape.com/catalogue/a-summer-i...,44.34,In stock,2,1
4,7,The Great Railway Bazaar,http://books.toscrape.com/catalogue/the-great-...,30.54,In stock,1,1
...,...,...,...,...,...,...,...
567,998,Amid the Chaos,http://books.toscrape.com/catalogue/amid-the-c...,36.58,In stock,1,48
568,1001,Sapiens: A Brief History of Humankind,#,49.99,In stock,5,54
569,1002,"Guns, Germs, and Steel",#,39.50,In stock,4,54
570,1003,The Silk Roads,#,35.00,In stock,4,54


#### 04) Write a sql query to get books that has price between 10 and 40 and has rate 3 or 4 or 1.

In [38]:
qry = '''
SELECT * FROM books_catalog 
WHERE price BETWEEN 10 AND 40 AND rating IN(3 , 4);
'''
cur.execute(qry)
cur.fetchall()

[(5,
  'Under the Tuscan Sun',
  'http://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html',
  Decimal('37.33'),
  'In stock',
  3,
  1),
 (10,
  'Neither Here nor There: Travels in Europe',
  'http://books.toscrape.com/catalogue/neither-here-nor-there-travels-in-europe_198/index.html',
  Decimal('38.95'),
  'In stock',
  3,
  1),
 (13,
  'In a Dark, Dark Wood',
  'http://books.toscrape.com/catalogue/in-a-dark-dark-wood_963/index.html',
  Decimal('19.63'),
  'In stock',
  3,
  2),
 (15,
  'A Murder in Time',
  'http://books.toscrape.com/catalogue/a-murder-in-time_877/index.html',
  Decimal('16.64'),
  'In stock',
  3,
  2),
 (18,
  'That Darkness (Gardiner and Renner #1)',
  'http://books.toscrape.com/catalogue/that-darkness-gardiner-and-renner-1_743/index.html',
  Decimal('13.92'),
  'In stock',
  3,
  2),
 (19,
  'Tastes Like Fear (DI Marnie Rome #3)',
  'http://books.toscrape.com/catalogue/tastes-like-fear-di-marnie-rome-3_742/index.html',
  Decimal('10.69'),
  'In st

In [39]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,5,Under the Tuscan Sun,http://books.toscrape.com/catalogue/under-the-...,37.33,In stock,3,1
1,10,Neither Here nor There: Travels in Europe,http://books.toscrape.com/catalogue/neither-he...,38.95,In stock,3,1
2,13,"In a Dark, Dark Wood",http://books.toscrape.com/catalogue/in-a-dark-...,19.63,In stock,3,2
3,15,A Murder in Time,http://books.toscrape.com/catalogue/a-murder-i...,16.64,In stock,3,2
4,18,That Darkness (Gardiner and Renner #1),http://books.toscrape.com/catalogue/that-darkn...,13.92,In stock,3,2
...,...,...,...,...,...,...,...
338,999,Dark Notes,http://books.toscrape.com/catalogue/dark-notes...,19.19,In stock,3,49
339,1000,The Long Shadow of Small Ghosts: Murder and Me...,http://books.toscrape.com/catalogue/the-long-s...,10.97,In stock,3,50
340,1002,"Guns, Germs, and Steel",#,39.50,In stock,4,54
341,1003,The Silk Roads,#,35.00,In stock,4,54


#### 05) Write a sql query to get the top 5 most expensive books (order books desc with price column and then limit the result to the first 5).

In [35]:
qry = '''
SELECT * FROM books_catalog 
order By price DESC
LIMIT 5;
'''
cur.execute(qry)
cur.fetchall()

[(192,
  'The Perfect Play (Play by Play #1)',
  'http://books.toscrape.com/catalogue/the-perfect-play-play-by-play-1_352/index.html',
  Decimal('59.99'),
  'In stock',
  3,
  7),
 (272,
  'Last One Home (New Beginnings #1)',
  'http://books.toscrape.com/catalogue/last-one-home-new-beginnings-1_383/index.html',
  Decimal('59.98'),
  'In stock',
  3,
  9),
 (926,
  'The Barefoot Contessa Cookbook',
  'http://books.toscrape.com/catalogue/the-barefoot-contessa-cookbook_440/index.html',
  Decimal('59.92'),
  'In stock',
  5,
  32),
 (369,
  'The Diary of a Young Girl',
  'http://books.toscrape.com/catalogue/the-diary-of-a-young-girl_634/index.html',
  Decimal('59.90'),
  'In stock',
  3,
  12),
 (965,
  'The Bone Hunters (Lexy Vaughan & Steven Macaulay #2)',
  'http://books.toscrape.com/catalogue/the-bone-hunters-lexy-vaughan-steven-macaulay-2_343/index.html',
  Decimal('59.71'),
  'In stock',
  3,
  36)]

In [36]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,192,The Perfect Play (Play by Play #1),http://books.toscrape.com/catalogue/the-perfec...,59.99,In stock,3,7
1,272,Last One Home (New Beginnings #1),http://books.toscrape.com/catalogue/last-one-h...,59.98,In stock,3,9
2,926,The Barefoot Contessa Cookbook,http://books.toscrape.com/catalogue/the-barefo...,59.92,In stock,5,32
3,369,The Diary of a Young Girl,http://books.toscrape.com/catalogue/the-diary-...,59.90,In stock,3,12
4,965,The Bone Hunters (Lexy Vaughan & Steven Macaul...,http://books.toscrape.com/catalogue/the-bone-h...,59.71,In stock,3,36


#### 06) Write a sql query to get the 3rd 10 books in the books table order first by rate desc and then by price asc.

In [33]:
qry = '''
SELECT * FROM books_catalog 
order By rating DESC  ,price 
LIMIT 10 OFFSET 20;
'''
cur.execute(qry)
cur.fetchall()

[(11,
  '1,000 Places to See Before You Die',
  'http://books.toscrape.com/catalogue/1000-places-to-see-before-you-die_1/index.html',
  Decimal('26.08'),
  'In stock',
  5,
  1),
 (311,
  'The Day the Crayons Came Home (Crayons)',
  'http://books.toscrape.com/catalogue/the-day-the-crayons-came-home-crayons_241/index.html',
  Decimal('26.33'),
  'In stock',
  5,
  10),
 (274,
  'Atlas Shrugged',
  'http://books.toscrape.com/catalogue/atlas-shrugged_314/index.html',
  Decimal('26.58'),
  'In stock',
  5,
  9),
 (423,
  'Gratitude',
  'http://books.toscrape.com/catalogue/gratitude_163/index.html',
  Decimal('26.66'),
  'In stock',
  5,
  12),
 (360,
  'Let It Out: A Journey Through Journaling',
  'http://books.toscrape.com/catalogue/let-it-out-a-journey-through-journaling_708/index.html',
  Decimal('26.79'),
  'In stock',
  5,
  12),
 (236,
  'The Time Keeper',
  'http://books.toscrape.com/catalogue/the-time-keeper_766/index.html',
  Decimal('27.88'),
  'In stock',
  5,
  9),
 (797,
  'Th

In [34]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,11,"1,000 Places to See Before You Die",http://books.toscrape.com/catalogue/1000-place...,26.08,In stock,5,1
1,311,The Day the Crayons Came Home (Crayons),http://books.toscrape.com/catalogue/the-day-th...,26.33,In stock,5,10
2,274,Atlas Shrugged,http://books.toscrape.com/catalogue/atlas-shru...,26.58,In stock,5,9
3,423,Gratitude,http://books.toscrape.com/catalogue/gratitude_...,26.66,In stock,5,12
4,360,Let It Out: A Journey Through Journaling,http://books.toscrape.com/catalogue/let-it-out...,26.79,In stock,5,12
5,236,The Time Keeper,http://books.toscrape.com/catalogue/the-time-k...,27.88,In stock,5,9
6,797,The Haters,http://books.toscrape.com/catalogue/the-haters...,27.89,In stock,5,20
7,630,Modern Romance,http://books.toscrape.com/catalogue/modern-rom...,28.26,In stock,5,17
8,57,The Passion of Dolssa,http://books.toscrape.com/catalogue/the-passio...,28.32,In stock,5,3
9,825,Les Fleurs du Mal,http://books.toscrape.com/catalogue/les-fleurs...,29.04,In stock,5,22


#### 07) Write a sql query to add a new category in categories table and add 5 books to this category in books table with title, rate and price.

In [15]:
qry = '''
INSERT INTO categories (category_name)
values(%s);
'''
cur.execute(qry , ('Art & Design',))

In [31]:
# Get the category_id of "Art & Design" First
qry = '''
SELECT category_id FROM categories WHERE category_name = 'Art & Design';
'''
cur.execute(qry)
category_id = cur.fetchone()[0]
category_id

54

In [32]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,category_id
0,54


In [21]:
# Insert 5 books linked to this category
books = [
    ('Sapiens: A Brief History of Humankind', 5, 49.99),
    ('Guns, Germs, and Steel', 4, 39.50),
    ('The Silk Roads', 4, 35.00),
    ('The Wright Brothers', 3, 28.90),
    ('The History of the Ancient World', 5, 41.75)
]

qry = '''
INSERT INTO books_catalog (title, url, price, availability, rating, category_id)
VALUES (%s, %s, %s, %s, %s, %s);
'''

for title , rating , price in books:
    cur.execute(qry , (title , '#', price, 'In stock', rating, category_id))

con.commit()

In [29]:
qry = '''
SELECT * FROM books_catalog
WHERE category_id = '54';
'''
cur.execute(qry)
cur.fetchall()

[(1001,
  'Sapiens: A Brief History of Humankind',
  '#',
  Decimal('49.99'),
  'In stock',
  5,
  54),
 (1002, 'Guns, Germs, and Steel', '#', Decimal('39.50'), 'In stock', 4, 54),
 (1003, 'The Silk Roads', '#', Decimal('35.00'), 'In stock', 4, 54),
 (1004, 'The Wright Brothers', '#', Decimal('28.90'), 'In stock', 3, 54),
 (1005,
  'The History of the Ancient World',
  '#',
  Decimal('41.75'),
  'In stock',
  5,
  54)]

In [30]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,1001,Sapiens: A Brief History of Humankind,#,49.99,In stock,5,54
1,1002,"Guns, Germs, and Steel",#,39.50,In stock,4,54
2,1003,The Silk Roads,#,35.00,In stock,4,54
3,1004,The Wright Brothers,#,28.90,In stock,3,54
4,1005,The History of the Ancient World,#,41.75,In stock,5,54


#### 08) Write a sql query to update book's rate to 3 that thier price < 20£.

In [3]:
qry = '''
UPDATE books_catalog
SET rating = 3
WHERE price < 20;
'''
cur.execute(qry)

#### 09) Write a sql query to delete all books that have price > 50£ and has rate <= 2.

In [27]:
qry = '''
DELETE FROM books_catalog
WHERE price > 50 AND rating <=2;
'''
cur.execute(qry)
con.commit()

#### 10) Write a sql query to count the number of books that have 'Secret' in thier names and price between 10£ and 25£.

In [25]:
qry = '''
SELECT COUNT(*) AS secret_book_count
FROM books_catalog
WHERE Title LIKE "%Secret%" AND price BETWEEN 10 AND 25;
'''
cur.execute(qry)
cur.fetchall()

[(5,)]

In [26]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,secret_book_count
0,5


#### 11) Write a sql query to get the minimum & maximum price for all the books that have rate 5.

In [23]:
qry = '''
SELECT MIN(price) AS min_price, MAX(price) AS max_price
FROM books_catalog
WHERE rating = 5;
'''
cur.execute(qry)
cur.fetchall()

[(Decimal('20.02'), Decimal('59.92'))]

In [24]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,min_price,max_price
0,20.02,59.92


#### 12) Write a sql query to calculate the avg price for all the books that have rate 5.

In [21]:
qry = '''
SELECT AVG(price) AS Avg_price
FROM books_catalog
WHERE rating = 5;
'''
cur.execute(qry)
cur.fetchall()

[(Decimal('41.111795'),)]

In [22]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,Avg_price
0,41.111795


#### 13) Write a sql query to sum all book's price that have rate 2 and price between 10 and 40

In [19]:
qry = '''
SELECT SUM(price) AS Sum_price
FROM books_catalog
WHERE rating = 2 AND price BETWEEN 10 AND 40;
'''
cur.execute(qry)
cur.fetchall()

[(Decimal('2772.75'),)]

In [20]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,Sum_price
0,2772.75


#### 14) Write a sql query to join both books & categories table into one new table containing book_name & category_name & book_rate and book_price.

In [17]:
qry = '''
SELECT b.title AS book_name , c.category_name , b.rating AS book_rate , b.price AS book_price
FROM books_catalog AS b INNER JOIN categories AS c
ON b.category_id = c.category_id;
'''
cur.execute(qry)
cur.fetchall()

[('Logan Kade (Fallen Crest High #5.5)', 'Academic', 3, Decimal('13.12')),
 ('The Torch Is Passed: A Harding Family Story',
  'Add a comment',
  3,
  Decimal('19.09')),
 ('The Mindfulness and Acceptance Workbook for Anxiety: A Guide to Breaking Free from Anxiety, Phobias, and Worry Using Acceptance and Commitment Therapy',
  'Add a comment',
  4,
  Decimal('23.89')),
 ('The Art Forger', 'Add a comment', 3, Decimal('40.76')),
 ('On a Midnight Clear', 'Add a comment', 3, Decimal('14.07')),
 ('Shobu Samurai, Project Aryoku (#3)', 'Add a comment', 3, Decimal('29.06')),
 ('Modern Romance', 'Add a comment', 5, Decimal('28.26')),
 ("The White Queen (The Cousins' War #1)",
  'Add a comment',
  5,
  Decimal('25.91')),
 ('The Song of Achilles', 'Add a comment', 5, Decimal('37.40')),
 ('The Immortal Life of Henrietta Lacks',
  'Add a comment',
  2,
  Decimal('40.67')),
 ('The Dovekeepers', 'Add a comment', 1, Decimal('48.78')),
 ('More Than Music (Chasing the Dream #1)',
  'Add a comment',
  2,
 

In [18]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_name,category_name,book_rate,book_price
0,Logan Kade (Fallen Crest High #5.5),Academic,3,13.12
1,The Torch Is Passed: A Harding Family Story,Add a comment,3,19.09
2,The Mindfulness and Acceptance Workbook for An...,Add a comment,4,23.89
3,The Art Forger,Add a comment,3,40.76
4,On a Midnight Clear,Add a comment,3,14.07
...,...,...,...,...
926,Boy Meets Boy,Young Adult,3,21.12
927,Annie on My Mind,Young Adult,5,36.83
928,New Moon (Twilight #2),Young Adult,3,12.86
929,The Haters,Young Adult,5,27.89


#### 15) Write a sql query to calculate how many books each rate has and have price between 20£ and 30£.

In [51]:
qry = '''
SELECT COUNT(*) AS book_count , rating 
FROM books_catalog
WHERE price BETWEEN 20 AND 30
GROUP BY rating
;
'''
cur.execute(qry)
cur.fetchall()

[(56, 1), (36, 5), (42, 3), (40, 2), (34, 4)]

In [52]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_count,rating
0,56,1
1,36,5
2,42,3
3,40,2
4,34,4


#### 16) Write a sql query to calculate how many books each category has having count > 10.

In [55]:
qry = '''
SELECT c.category_name, COUNT(b.book_id) AS book_count
FROM categories AS c
INNER JOIN books_catalog AS b
ON c.category_id = b.category_id
GROUP BY c.category_name
HAVING COUNT(b.book_id) > 10;
'''
cur.execute(qry)
cur.fetchall()

[('Add a comment', 60),
 ('Business', 11),
 ('Childrens', 28),
 ('Classics', 17),
 ('Default', 144),
 ('Fantasy', 40),
 ('Fiction', 60),
 ('Food and Drink', 28),
 ('Historical Fiction', 24),
 ('History', 16),
 ('Horror', 15),
 ('Music', 11),
 ('Mystery', 30),
 ('Nonfiction', 104),
 ('Poetry', 18),
 ('Romance', 33),
 ('Science', 13),
 ('Science Fiction', 14),
 ('Sequential Art', 70),
 ('Thriller', 11),
 ('Travel', 11),
 ('Womens Fiction', 16),
 ('Young Adult', 50)]

In [56]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,category_name,book_count
0,Add a comment,60
1,Business,11
2,Childrens,28
3,Classics,17
4,Default,144
5,Fantasy,40
6,Fiction,60
7,Food and Drink,28
8,Historical Fiction,24
9,History,16


#### 17) Write a sql query to get all books with category_name='Music' using subquery.	 

In [58]:
qry = '''
SELECT * 
FROM books_catalog
WHERE category_id IN(
SELECT category_id
FROM categories
WHERE category_name = 'Music'
);
'''
cur.execute(qry)
cur.fetchall()

[(438,
  'Rip it Up and Start Again',
  'http://books.toscrape.com/catalogue/rip-it-up-and-start-again_986/index.html',
  Decimal('35.02'),
  'In stock',
  5,
  13),
 (439,
  'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
  'http://books.toscrape.com/catalogue/our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html',
  Decimal('57.25'),
  'In stock',
  3,
  13),
 (440,
  'How Music Works',
  'http://books.toscrape.com/catalogue/how-music-works_979/index.html',
  Decimal('37.32'),
  'In stock',
  2,
  13),
 (441,
  'Love Is a Mix Tape (Music #1)',
  'http://books.toscrape.com/catalogue/love-is-a-mix-tape-music-1_711/index.html',
  Decimal('18.03'),
  'In stock',
  3,
  13),
 (442,
  'Please Kill Me: The Uncensored Oral History of Punk',
  'http://books.toscrape.com/catalogue/please-kill-me-the-uncensored-oral-history-of-punk_537/index.html',
  Decimal('31.19'),
  'In stock',
  4,
  13),
 (443,
  "Kill 'Em and

In [59]:
df = pd.read_sql(qry , con)
df

C:\Users\Alaa\AppData\Local\Temp\ipykernel_48364\4063385464.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry , con)


,book_id,title,url,price,availability,rating,category_id
0,438,Rip it Up and Start Again,http://books.toscrape.com/catalogue/rip-it-up-...,35.02,In stock,5,13
1,439,Our Band Could Be Your Life: Scenes from the A...,http://books.toscrape.com/catalogue/our-band-c...,57.25,In stock,3,13
2,440,How Music Works,http://books.toscrape.com/catalogue/how-music-...,37.32,In stock,2,13
3,441,Love Is a Mix Tape (Music #1),http://books.toscrape.com/catalogue/love-is-a-...,18.03,In stock,3,13
4,442,Please Kill Me: The Uncensored Oral History of...,http://books.toscrape.com/catalogue/please-kil...,31.19,In stock,4,13
5,443,Kill 'Em and Leave: Searching for James Brown ...,http://books.toscrape.com/catalogue/kill-em-an...,45.05,In stock,5,13
6,445,This Is Your Brain on Music: The Science of a ...,http://books.toscrape.com/catalogue/this-is-yo...,38.40,In stock,1,13
7,446,Orchestra of Exiles: The Story of Bronislaw Hu...,http://books.toscrape.com/catalogue/orchestra-...,12.36,In stock,3,13
8,447,No One Here Gets Out Alive,http://books.toscrape.com/catalogue/no-one-her...,20.02,In stock,5,13
9,448,Life,http://books.toscrape.com/catalogue/life_104/i...,31.58,In stock,5,13
